In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/grid2_u_10000_1/"

"../../graphs/grid2_u_10000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:5
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

@time for i in 4:5
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  1.043294 seconds (365.44 k allocations: 17.987 MB, 1.00% gc time)
  0.507582 seconds (267.65 k allocations: 7.725 MB, 1.52% gc time)
  3.747604 seconds (32.08 M allocations: 839.232 MB, 3.52% gc time)
Working on iteration 10 with a-norm error 4074.0722970023944
Working on iteration 20 with a-norm error 3169.9976112623312
Working on iteration 30 with a-norm error 2388.73545645245
Working on iteration 40 with a-norm error 1596.869983337341
Working on iteration 50 with a-norm error 1105.5311946631139
Working on iteration 60 with a-norm error 761.7390359751378
Working on iteration 70 with a-norm error 560.4290615014557
Working on iteration 80 with a-norm error 409.7169664379145
Working on iteration 90 with a-norm error 287.9059209640351
Working on iteration 100 with a-norm error 207.00299574477066
Working on iteration 110 with a-norm error 161.03049549327204
Working on iteration 120 with a-norm error 116.75188165414124
Working on iteration 130 with a-norm error 90.1457635022925
Working o